In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train1 = pd.read_csv('/home/user3/anaconda3/빅콘테스트/train_activity.csv', engine = 'python')
train2 = pd.read_csv('/home/user3/anaconda3/빅콘테스트/train_combat.csv', engine = 'python')
train3 = pd.read_csv('/home/user3/anaconda3/빅콘테스트/train_pledge.csv', engine = 'python')
train4 = pd.read_csv('/home/user3/anaconda3/빅콘테스트/train_payment.csv', engine = 'python')
train5 = pd.read_csv('/home/user3/anaconda3/빅콘테스트/train_trade.csv', engine = 'python')
y_train = pd.read_csv('/home/user3/anaconda3/빅콘테스트/train_label.csv', engine = 'python')

In [3]:
# 잔존 고객 0, 이탈 고객 1
y_train['leave'] = 0
y_train['leave'][y_train.survival_time < 64] = 1

/home/user3/.conda/envs/hun/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
# 결제 기록이 없으면 0, 결제 기록이 있으면 1
y_train['amount'] = 0
y_train['amount'][y_train.amount_spent > 0] = 1

/home/user3/.conda/envs/hun/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.25, random_state = 42)
for train_index, test_index in split.split(y_train, y_train['leave']):
    strat_y_train = y_train.loc[train_index]
    strat_y_test = y_train.loc[test_index]

In [6]:
len(strat_y_train), len(strat_y_test)

(30000, 10000)

In [7]:
strat_y_train['leave'].mean(), strat_y_test['leave'].mean()

(0.4501, 0.4501)

In [8]:
y_train = strat_y_train.sort_values(['acc_id'], ascending = True)
y_test = strat_y_test.sort_values(['acc_id'], ascending = True)

In [9]:
train_id = y_train['acc_id'].values
test_id = y_test['acc_id'].values

In [10]:
pd.DataFrame(y_train['acc_id']).to_csv('train_index.csv', index = False)
pd.DataFrame(y_test['acc_id']).to_csv('test_index.csv', index = False)

# 1. 전처리

### 전처리에 필요한 함수 정의

In [11]:
def divide_by_week(data, variable, user_id):
    '''
    데이터와 변수를 넣으면 각 유저 아이디에 대한 주차별 변수의 합을 구함
    '''
    
    week_1 = np.zeros(len(user_id))
    week_2 = np.zeros(len(user_id))
    week_3 = np.zeros(len(user_id))
    week_4 = np.zeros(len(user_id))
    for i in range(0, len(user_id)):
        data_sub = data[data['acc_id'] == user_id[i]]
        week_1[i] = data_sub[(data_sub['day'] >= 1) & (data_sub['day'] <= 7)][variable].sum()
        week_2[i] = data_sub[(data_sub['day'] >= 8) & (data_sub['day'] <= 14)][variable].sum()
        week_3[i] = data_sub[(data_sub['day'] >= 15) & (data_sub['day'] <= 21)][variable].sum()
        week_4[i] = data_sub[(data_sub['day'] >= 22) & (data_sub['day'] <= 28)][variable].sum()
    print(variable, 'done')
        
    return week_1, week_2, week_3, week_4

In [12]:
def pledge_divide_by_week(data, variable, pledge_id):
    '''
    데이터와 변수를 넣으면 각 혈맹 아이디에 대한 주차별 변수의 평균을 구함
    '''
    
    week_1 = np.zeros(len(pledge_id))
    week_2 = np.zeros(len(pledge_id))
    week_3 = np.zeros(len(pledge_id))
    week_4 = np.zeros(len(pledge_id))
    for i in range(0, len(pledge_id)):
        data_sub = data[data['pledge_id'] == pledge_id[i]]
        week_1[i] = data_sub[(data_sub['day'] >= 1) & (data_sub['day'] <= 7)][variable].mean()
        week_2[i] = data_sub[(data_sub['day'] >= 8) & (data_sub['day'] <= 14)][variable].mean()
        week_3[i] = data_sub[(data_sub['day'] >= 15) & (data_sub['day'] <= 21)][variable].mean()
        week_4[i] = data_sub[(data_sub['day'] >= 22) & (data_sub['day'] <= 28)][variable].mean()
    print(variable, 'done')
        
    return week_1, week_2, week_3, week_4

In [13]:
def char_divide_by_week(data, variable, char_id):
    '''
    데이터와 변수를 넣으면 각 캐릭터 아이디에 대한 주차별 변수의 평균을 구함
    '''
    
    week_1 = np.zeros(len(char_id))
    week_2 = np.zeros(len(char_id))
    week_3 = np.zeros(len(char_id))
    week_4 = np.zeros(len(char_id))
    for i in range(0, len(char_id)):
        data_sub = data[data['char_id'] == char_id[i]]
        week_1[i] = data_sub[(data_sub['day'] >= 1) & (data_sub['day'] <= 7)][variable].mean()
        week_2[i] = data_sub[(data_sub['day'] >= 8) & (data_sub['day'] <= 14)][variable].mean()
        week_3[i] = data_sub[(data_sub['day'] >= 15) & (data_sub['day'] <= 21)][variable].mean()
        week_4[i] = data_sub[(data_sub['day'] >= 22) & (data_sub['day'] <= 28)][variable].mean()
    print(variable, 'done')
        
    return week_1, week_2, week_3, week_4

In [14]:
def preprocess(data1, data2, data3, data4, data5, user_id):
    '''
    유저와 케릭터에 대한 28일 동안의 모든 활동 데이터를 각 유저에 대한 정보로 전처리
    data1 : 기본 활동 데이터(DataFrame)
    data2 : 전투 데이터(DataFrame)
    data3 : 혈맹 데이터(DataFrame)
    data4 : 결제 데이터(DataFrame)
    data5 : 거래 데이터(DataFrame)
    user_id(=acc_id) : 유저별 id(np.array)
    '''
    
    # user_id에 해당하는 행만 subset
    index_list = []
    for i in range(0, len(data1)):
        if(data1['acc_id'][i] in user_id):
            index_list.append(i)            
    data1_sub = data1.loc[index_list]
    
    index_list = []
    for i in range(0, len(data2)):
        if(data2['acc_id'][i] in user_id):
            index_list.append(i)            
    data2_sub = data2.loc[index_list]
    
    index_list = []
    for i in range(0, len(data3)):
        if(data3['acc_id'][i] in user_id):
            index_list.append(i)            
    data3_sub = data3.loc[index_list]
    
    index_list = []
    for i in range(0, len(data4)):
        if(data4['acc_id'][i] in user_id):
            index_list.append(i)            
    data4_sub = data4.loc[index_list]
    
    index_list = []
    for i in range(0, len(data5)):
        if(data5['source_acc_id'][i] in user_id):
            index_list.append(i)      
    source_data = data5.loc[index_list]
    
    index_list = []
    for i in range(0, len(data5)):
        if(data5['target_acc_id'][i] in user_id):
            index_list.append(i)      
    target_data = data5.loc[index_list]
    
    
    # 연속형 변수명만 추출
    data1_col = data1.columns[4:]
    data2_col = data2.columns[6:]
    data3_col = data3.columns[5:-1]
    
    
    # 각 유저에 대한 연속형 변수를 주차별 합으로 새로운 변수 만듦
    dic = {}  # 새 변수를 담기 위한 빈 dictionary

    for var_name in data1_col:
        week_1, week_2, week_3, week_4 = divide_by_week(data = data1_sub, variable = var_name, user_id = user_id)
        key_1 = var_name + '_1'
        key_2 = var_name + '_2'
        key_3 = var_name + '_3'
        key_4 = var_name + '_4'
        dic[key_1] = week_1
        dic[key_2] = week_2
        dic[key_3] = week_3
        dic[key_4] = week_4
        
    for var_name in data2_col:
        week_1, week_2, week_3, week_4 = divide_by_week(data = data2_sub, variable = var_name, user_id = user_id)
        key_1 = 'combat_' + var_name + '_1'
        key_2 = 'combat_' + var_name + '_2'
        key_3 = 'combat_' + var_name + '_3'
        key_4 = 'combat_' + var_name + '_4'
        dic[key_1] = week_1
        dic[key_2] = week_2
        dic[key_3] = week_3
        dic[key_4] = week_4
    
    
    # user_id와 연속형 변수들을 concatenate 하여 data_array로 저장
    user_id_clone = user_id.copy()
    data_array = user_id_clone.reshape(-1, 1)

    for key in dic.keys():
        data_array = np.concatenate([data_array, dic[key].reshape(-1, 1)], axis = 1)
        
        
    # data_array를 데이터프레임 형태로 바꿔주고 컬럼명 지정
    colnames = ['acc_id']
    colnames.extend(list(dic.keys())[0:])

    data = pd.DataFrame(data_array)
    data.columns = colnames
    
    
    # 혈맹 데이터를 이용해 혈맹 군집화
    dic = {}
    char_list = []
    char_matrix = data3_sub.groupby(['acc_id', 'char_id']).count()['day']
    data3_id = data3_sub['acc_id'].unique()
    for i in data3_id:
        char_list.append(np.argmax(char_matrix[i]))
    char_id = np.array(char_list)

    for var_name in data3_col:
        week_1, week_2, week_3, week_4 = char_divide_by_week(data = data3_sub, variable = var_name, char_id = char_id)
        key_1 = var_name + '_1'
        key_2 = var_name + '_2'
        key_3 = var_name + '_3'
        key_4 = var_name + '_4'
        dic[key_1] = week_1
        dic[key_2] = week_2
        dic[key_3] = week_3
        dic[key_4] = week_4
        
    char_id_clone = char_id.copy()
    char_data_array = np.concatenate([data3_id.reshape(-1, 1), char_id_clone.reshape(-1, 1)], axis = 1)

    for key in dic.keys():
        char_data_array = np.concatenate([char_data_array, dic[key].reshape(-1, 1)], axis = 1)
        
    colnames = ['acc_id', 'char_id']
    colnames.extend(list(dic.keys())[0:])

    char_data = pd.DataFrame(char_data_array)
    char_data.columns = colnames
    char_data = char_data.fillna(0)
    
    char_data['pledge_group'] = kmean.predict(char_data.iloc[:, 2:]) + 1
    char_data['pledge_group'] = char_data['pledge_group'].astype('int32')
    data = pd.merge(data, char_data.loc[:, ['acc_id', 'pledge_group']], how = 'left', on = 'acc_id')
    data = data.fillna(0)
    
    
    # 결제 데이터에서 28일 동안 결제 횟수, 1회 결제시 결제 금액 추출    
    amount_cnt = data4_sub.groupby('acc_id')['amount_spent'].count()
    amount_mean = data4_sub.groupby('acc_id')['amount_spent'].sum() / data4_sub.groupby('acc_id')['amount_spent'].count()
    acc_id_sorted = np.sort(data4_sub['acc_id'].unique())
    
    amount_array = np.concatenate([acc_id_sorted.reshape(-1, 1), amount_cnt.values.reshape(-1, 1),
                                  amount_mean.values.reshape(-1, 1)], axis = 1)
    amount_data = pd.DataFrame(amount_array)
    amount_data.columns = ['acc_id', 'amount_cnt', 'amount_mean']
    
    data = pd.merge(data, amount_data, how = 'left', on = 'acc_id')
    data = data.fillna(0)  # 결제 기록이 없어 nan인 값을 모두 0으로 대체
    
    
    # 기본 활동 데이터의 서버를 가장 플레이 시간이 많은 케릭터 기준으로 추출
    char_list = []
    char_matrix = data1_sub.groupby(['acc_id', 'char_id'])['playtime'].sum()
    for i in user_id:
        char_list.append(np.argmax(char_matrix[i]))
        
    server_list = []
    for i in char_list:
        user_char = data1_sub[data1_sub['char_id'] == i]
        char_server = user_char['server'].values[-1]
        server_list.append(char_server)
    
    server_array = np.concatenate([np.array(user_id).reshape(-1, 1), np.array(server_list).reshape(-1, 1)], axis = 1)
    server_data = pd.DataFrame(server_array)
    server_data.columns = ['acc_id', 'server']
    server_data['acc_id'] = server_data['acc_id'].astype('float64')
    data = pd.merge(data, server_data, how = 'left', on = 'acc_id')
    
    # 0은 특수 서버, 1은 일반 서버, 2는 non pvp 서버
    data['server_group'] = 0
    data['server_group'][(data['server'] == 'aa') | (data['server'] == 'ab') | (data['server'] == 'ac') |
                         (data['server'] == 'ad') | (data['server'] == 'ae') | (data['server'] == 'af') |
                         (data['server'] == 'ag') | (data['server'] == 'ah') | (data['server'] == 'ai') |
                         (data['server'] == 'aj') | (data['server'] == 'ak') | (data['server'] == 'al') |
                         (data['server'] == 'am') | (data['server'] == 'an') | (data['server'] == 'ao') |
                         (data['server'] == 'ap') | (data['server'] == 'aq') | (data['server'] == 'ar') |
                         (data['server'] == 'as') | (data['server'] == 'at') | (data['server'] == 'au') |
                         (data['server'] == 'av') | (data['server'] == 'aw') | (data['server'] == 'ax') |
                         (data['server'] == 'ay') | (data['server'] == 'az') | (data['server'] == 'ba') |
                         (data['server'] == 'bb') | (data['server'] == 'bc') | (data['server'] == 'bd')] = 1
    data['server_group'][(data['server'] == 'bi') | (data['server'] == 'bs') | (data['server'] == 'bg')] = 2
    data = data.drop('server', axis = 1)
    
    
    # 전투 데이터의 직업, 레벨 변수를 가장 접속일 수가 많은 케릭터 기준으로 추출    
    char_list = []
    char_matrix = data2_sub.groupby(['acc_id', 'char_id']).count()['day']
    for i in user_id:
        char_list.append(np.argmax(char_matrix[i]))
        
    class_list = []
    level_list = []
    for i in char_list:
        user_char = data2_sub[data2_sub['char_id'] == i]
        char_class = user_char['class'].values[-1]
        char_level = user_char['level'].values[-1]
        class_list.append(char_class)
        level_list.append(char_level)
    
    char_array = np.concatenate([np.array(user_id).reshape(-1, 1), np.array(class_list).reshape(-1, 1),
                                 np.array(level_list).reshape(-1, 1)], axis = 1)
    char_data = pd.DataFrame(char_array)
    char_data.columns = ['acc_id', 'class', 'level']
    char_data['acc_id'] = char_data['acc_id'].astype('float64')
    data = pd.merge(char_data, data, how = 'left', on = 'acc_id')
    
    
    # 범주형 변수 one-hot encoding
    server_group_dummies = pd.get_dummies(data['server_group']).rename(columns = lambda x : 'server_group_' + str(x))
    class_dummies = pd.get_dummies(data['class']).rename(columns = lambda x : 'class_' + str(x))
    pledge_group_dummies = pd.get_dummies(data['pledge_group']).rename(columns = lambda x : 'pledge_group_' + str(x))

    data = pd.concat([data, server_group_dummies, class_dummies, pledge_group_dummies], axis = 1)
    data.drop(['server_group', 'class', 'pledge_group'], axis = 1, inplace = True)
    
    
    # 거래 데이터를 user_id에 대하여 판매, 구매로 구분
    trade_data = pd.DataFrame(user_id)
    trade_data.columns = ['acc_id']
    
    source_data_type_0 = source_data[source_data['type'] == 0]
    source_data_type_1 = source_data[source_data['type'] == 1]
    
    for i in range(1, 5):
        source_data_type_0_week = source_data_type_0[(source_data_type_0['day'] >= (7 * i - 6)) & (source_data_type_0['day'] <= (7 * i))]
        source_data_type_0_week_sum = source_data_type_0_week.groupby(['source_acc_id', 'item_type'])['item_price'].sum().unstack('item_type')
        source_data_type_0_week_sum = source_data_type_0_week_sum.fillna(0)
        source_data_type_0_week_sum = source_data_type_0_week_sum.reset_index(level = 'source_acc_id')
        source_data_type_0_week_sum = source_data_type_0_week_sum.rename(columns = {'source_acc_id' : 'acc_id'})
        col = 'source_type_0_week'+ str(i) + '_' + source_data_type_0_week_sum.columns[1:]
        source_data_type_0_week_sum.columns = source_data_type_0_week_sum.columns[0:1].append(col)
        trade_data = pd.merge(trade_data, source_data_type_0_week_sum, how = 'left', on = 'acc_id')
    
    for i in range(1, 5):
        source_data_type_1_week = source_data_type_1[(source_data_type_1['day'] >= (7 * i - 6)) & (source_data_type_1['day'] <= (7 * i))]
        source_data_type_1_week_sum = source_data_type_1_week.groupby(['source_acc_id', 'item_type'])['item_amount'].sum().unstack('item_type')
        source_data_type_1_week_sum = source_data_type_1_week_sum.fillna(0)
        source_data_type_1_week_sum = source_data_type_1_week_sum.reset_index(level = 'source_acc_id')
        source_data_type_1_week_sum = source_data_type_1_week_sum.rename(columns = {'source_acc_id' : 'acc_id'})
        col = 'source_type_1_week'+ str(i) + '_' + source_data_type_1_week_sum.columns[1:]
        source_data_type_1_week_sum.columns = source_data_type_1_week_sum.columns[0:1].append(col)
        trade_data = pd.merge(trade_data, source_data_type_1_week_sum, how = 'left', on = 'acc_id')
        
    target_data_type_0 = target_data[target_data['type'] == 0]
    target_data_type_1 = target_data[target_data['type'] == 1]
    
    for i in range(1, 5):
        target_data_type_0_week = target_data_type_0[(target_data_type_0['day'] >= (7 * i - 6)) & (target_data_type_0['day'] <= (7 * i))]
        target_data_type_0_week_sum = target_data_type_0_week.groupby(['target_acc_id', 'item_type'])['item_price'].sum().unstack('item_type')
        target_data_type_0_week_sum = target_data_type_0_week_sum.fillna(0)
        target_data_type_0_week_sum = target_data_type_0_week_sum.reset_index(level = 'target_acc_id')
        target_data_type_0_week_sum = target_data_type_0_week_sum.rename(columns = {'target_acc_id' : 'acc_id'})
        col = 'target_type_0_week'+ str(i) + '_' + target_data_type_0_week_sum.columns[1:]
        target_data_type_0_week_sum.columns = target_data_type_0_week_sum.columns[0:1].append(col)
        trade_data = pd.merge(trade_data, target_data_type_0_week_sum, how = 'left', on = 'acc_id')
        
    for i in range(1, 5):
        target_data_type_1_week = target_data_type_1[(target_data_type_1['day'] >= (7 * i - 6)) & (target_data_type_1['day'] <= (7 * i))]
        target_data_type_1_week_sum = target_data_type_1_week.groupby(['target_acc_id', 'item_type'])['item_amount'].sum().unstack('item_type')
        target_data_type_1_week_sum = target_data_type_1_week_sum.fillna(0)
        target_data_type_1_week_sum = target_data_type_1_week_sum.reset_index(level = 'target_acc_id')
        target_data_type_1_week_sum = target_data_type_1_week_sum.rename(columns = {'target_acc_id' : 'acc_id'})
        col = 'target_type_1_week'+ str(i) + '_' + target_data_type_1_week_sum.columns[1:]
        target_data_type_1_week_sum.columns = target_data_type_1_week_sum.columns[0:1].append(col)
        trade_data = pd.merge(trade_data, target_data_type_1_week_sum, how = 'left', on = 'acc_id')
        
    trade_data = trade_data.fillna(0)
    
    
    # 주요 거래 아이템(아데나, 강화 주문서, 기타) 외 아이템은 거래 여부에 따라 0, 1로 코딩
    main_item = ['adena', 'enchant_scroll', 'etc']
    item_list = []
     
    for item in main_item:
        for i in range(0, len(trade_data.columns)):
            if(item in trade_data.columns[i]):
                item_list.append(i)                
    trade_data_sub = trade_data.iloc[:, item_list]
    
    weapon_list = []
    for i in range(0, len(trade_data.columns)):
        if('weapon' in trade_data.columns[i]):
            weapon_list.append(i)       
    trade_data_sub['source_weapon'] = 0
    trade_data_sub['source_weapon'][trade_data.iloc[:, weapon_list].iloc[:, :8].sum(axis = 1) > 0] = 1
    trade_data_sub['target_weapon'] = 0
    trade_data_sub['target_weapon'][trade_data.iloc[:, weapon_list].iloc[:, 8:].sum(axis = 1) > 0] = 1
    
    armor_list = []
    for i in range(0, len(trade_data.columns)):
        if('armor' in trade_data.columns[i]):
            armor_list.append(i)     
    trade_data_sub['source_armor'] = 0
    trade_data_sub['source_armor'][trade_data.iloc[:, armor_list].iloc[:, :8].sum(axis = 1) > 0] = 1
    trade_data_sub['target_armor'] = 0
    trade_data_sub['target_armor'][trade_data.iloc[:, armor_list].iloc[:, 8:].sum(axis = 1) > 0] = 1
    
    accessory_list = []
    for i in range(0, len(trade_data.columns)):
        if('accessory' in trade_data.columns[i]):
            accessory_list.append(i)    
    trade_data_sub['source_accessory'] = 0
    trade_data_sub['source_accessory'][trade_data.iloc[:, accessory_list].iloc[:, :8].sum(axis = 1) > 0] = 1
    trade_data_sub['target_accessory'] = 0
    trade_data_sub['target_accessory'][trade_data.iloc[:, accessory_list].iloc[:, 8:].sum(axis = 1) > 0] = 1
    
    spell_list = []
    for i in range(0, len(trade_data.columns)):
        if('spell' in trade_data.columns[i]):
            spell_list.append(i)     
    trade_data_sub['source_spell'] = 0
    trade_data_sub['source_spell'][trade_data.iloc[:, spell_list].iloc[:, :8].sum(axis = 1) > 0] = 1
    trade_data_sub['target_spell'] = 0
    trade_data_sub['target_spell'][trade_data.iloc[:, spell_list].iloc[:, 8:].sum(axis = 1) > 0] = 1
    
    trade_data_sub['acc_id'] = trade_data['acc_id']
    
    data = pd.merge(data, trade_data_sub, how = 'left', on = 'acc_id')
        
    
    return data

### 혈맹에 대한 군집 형성

In [15]:
index_list = []
for i in range(0, len(train3)):
    if(train3['acc_id'][i] in train_id):
        index_list.append(i)            
train3_sub = train3.loc[index_list]

In [16]:
dic = {}
train3_col = train3_sub.columns[5:-1]
pledge_id = train3_sub['pledge_id'].unique()

for var_name in train3_col:
    week_1, week_2, week_3, week_4 = pledge_divide_by_week(data = train3_sub, variable = var_name, pledge_id = pledge_id)
    key_1 = var_name + '_1'
    key_2 = var_name + '_2'
    key_3 = var_name + '_3'
    key_4 = var_name + '_4'
    dic[key_1] = week_1
    dic[key_2] = week_2
    dic[key_3] = week_3
    dic[key_4] = week_4

play_char_cnt done
combat_char_cnt done
pledge_combat_cnt done
random_attacker_cnt done
random_defender_cnt done
same_pledge_cnt done
temp_cnt done
etc_cnt done
combat_play_time done


In [17]:
pledge_id_clone = pledge_id.copy()
pledge_data_array = pledge_id_clone.reshape(-1, 1)

for key in dic.keys():
    pledge_data_array = np.concatenate([pledge_data_array, dic[key].reshape(-1, 1)], axis = 1)
        
colnames = ['pledge_id']
colnames.extend(list(dic.keys())[0:])

pledge_data = pd.DataFrame(pledge_data_array)
pledge_data.columns = colnames
pledge_data = pledge_data.fillna(0)

In [18]:
from sklearn.cluster import KMeans

X = pledge_data.iloc[:, 1:]
kmean = KMeans(n_clusters = 3, random_state = 42, n_jobs = -1)
kmean.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=-1, precompute_distances='auto',
       random_state=42, tol=0.0001, verbose=0)

### raw data -> 전처리 데이터 변환

In [19]:
X_train = preprocess(train1, train2, train3, train4, train5, train_id)

playtime done
npc_kill done
solo_exp done
party_exp done
quest_exp done
rich_monster done
death done
revive done
exp_recovery done
fishing done
private_shop done
game_money_change done
enchant_count done
pledge_cnt done
random_attacker_cnt done
random_defender_cnt done
temp_cnt done
same_pledge_cnt done
etc_cnt done
num_opponent done


/home/user3/.conda/envs/hun/lib/python3.6/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


play_char_cnt done
combat_char_cnt done
pledge_combat_cnt done
random_attacker_cnt done
random_defender_cnt done
same_pledge_cnt done
temp_cnt done
etc_cnt done
combat_play_time done


/home/user3/.conda/envs/hun/lib/python3.6/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)
/home/user3/.conda/envs/hun/lib/python3.6/site-packages/ipykernel_launcher.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/user3/.conda/envs/hun/lib/python3.6/site-packages/ipykernel_launcher.py:181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexin

In [20]:
X_train.head()

,acc_id,level,playtime_1,playtime_2,playtime_3,playtime_4,npc_kill_1,npc_kill_2,npc_kill_3,npc_kill_4,...,target_type_1_week3_etc,target_type_1_week4_etc,source_weapon,target_weapon,source_armor,target_armor,source_accessory,target_accessory,source_spell,target_spell
0,2.0,0,22.919696,22.980553,22.659883,22.973531,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0,0,1,0,0,0,0,0
1,5.0,15,0.000000,0.589845,0.213000,5.313287,0.000000,0.310229,0.162887,2.763336,...,0.000000e+00,0.000000,0,0,0,0,0,0,0,0
2,8.0,16,4.987936,4.889629,4.653223,4.267014,26.808113,1.325063,1.444694,1.594739,...,0.000000e+00,0.000000,0,0,0,0,0,0,1,0
3,17.0,16,5.406914,9.048973,7.831832,6.176989,25.931158,1.328780,1.362575,1.834339,...,7.190952e-08,0.000029,1,1,0,1,1,1,0,1
4,21.0,16,6.048253,7.363701,4.966870,3.452466,4.379366,4.602406,0.654252,1.135141,...,0.000000e+00,0.000000,1,0,0,1,0,1,0,0


In [21]:
X_train.columns.values

array(['acc_id', 'level', 'playtime_1', 'playtime_2', 'playtime_3',
       'playtime_4', 'npc_kill_1', 'npc_kill_2', 'npc_kill_3',
       'npc_kill_4', 'solo_exp_1', 'solo_exp_2', 'solo_exp_3',
       'solo_exp_4', 'party_exp_1', 'party_exp_2', 'party_exp_3',
       'party_exp_4', 'quest_exp_1', 'quest_exp_2', 'quest_exp_3',
       'quest_exp_4', 'rich_monster_1', 'rich_monster_2',
       'rich_monster_3', 'rich_monster_4', 'death_1', 'death_2',
       'death_3', 'death_4', 'revive_1', 'revive_2', 'revive_3',
       'revive_4', 'exp_recovery_1', 'exp_recovery_2', 'exp_recovery_3',
       'exp_recovery_4', 'fishing_1', 'fishing_2', 'fishing_3',
       'fishing_4', 'private_shop_1', 'private_shop_2', 'private_shop_3',
       'private_shop_4', 'game_money_change_1', 'game_money_change_2',
       'game_money_change_3', 'game_money_change_4', 'enchant_count_1',
       'enchant_count_2', 'enchant_count_3', 'enchant_count_4',
       'combat_pledge_cnt_1', 'combat_pledge_cnt_2',
       'combat

In [22]:
X_test = preprocess(train1, train2, train3, train4, train5, test_id)

playtime done
npc_kill done
solo_exp done
party_exp done
quest_exp done
rich_monster done
death done
revive done
exp_recovery done
fishing done
private_shop done
game_money_change done
enchant_count done
pledge_cnt done
random_attacker_cnt done
random_defender_cnt done
temp_cnt done
same_pledge_cnt done
etc_cnt done
num_opponent done
play_char_cnt done
combat_char_cnt done
pledge_combat_cnt done
random_attacker_cnt done
random_defender_cnt done
same_pledge_cnt done
temp_cnt done
etc_cnt done
combat_play_time done


/home/user3/.conda/envs/hun/lib/python3.6/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)
/home/user3/.conda/envs/hun/lib/python3.6/site-packages/ipykernel_launcher.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/user3/.conda/envs/hun/lib/python3.6/site-packages/ipykernel_launcher.py:181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexin

In [23]:
X_test.head()

,acc_id,level,playtime_1,playtime_2,playtime_3,playtime_4,npc_kill_1,npc_kill_2,npc_kill_3,npc_kill_4,...,target_type_1_week3_etc,target_type_1_week4_etc,source_weapon,target_weapon,source_armor,target_armor,source_accessory,target_accessory,source_spell,target_spell
0,20.0,17,5.526287,7.014944,3.508642,9.538170,27.285960,1.631575,1.581222,3.601428,...,0.719095,0.000000,0,0,0,1,1,1,0,0
1,62.0,16,8.983435,11.438782,7.651602,9.952466,27.273118,0.949612,0.734682,1.747826,...,0.000000,0.000000,0,1,0,0,0,1,0,0
2,81.0,16,7.007922,12.358660,11.382606,10.820849,29.873229,4.915677,3.930919,4.844371,...,0.000225,0.000002,1,1,0,1,0,0,0,1
3,86.0,0,22.800322,22.971190,22.510081,22.964168,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1,0,1,0,1,0,1,1
4,125.0,16,0.000000,0.414296,0.154483,0.482175,0.000000,0.088540,0.023994,0.487648,...,0.000000,0.000000,0,0,0,1,0,0,0,0


In [24]:
X_test.columns.values

array(['acc_id', 'level', 'playtime_1', 'playtime_2', 'playtime_3',
       'playtime_4', 'npc_kill_1', 'npc_kill_2', 'npc_kill_3',
       'npc_kill_4', 'solo_exp_1', 'solo_exp_2', 'solo_exp_3',
       'solo_exp_4', 'party_exp_1', 'party_exp_2', 'party_exp_3',
       'party_exp_4', 'quest_exp_1', 'quest_exp_2', 'quest_exp_3',
       'quest_exp_4', 'rich_monster_1', 'rich_monster_2',
       'rich_monster_3', 'rich_monster_4', 'death_1', 'death_2',
       'death_3', 'death_4', 'revive_1', 'revive_2', 'revive_3',
       'revive_4', 'exp_recovery_1', 'exp_recovery_2', 'exp_recovery_3',
       'exp_recovery_4', 'fishing_1', 'fishing_2', 'fishing_3',
       'fishing_4', 'private_shop_1', 'private_shop_2', 'private_shop_3',
       'private_shop_4', 'game_money_change_1', 'game_money_change_2',
       'game_money_change_3', 'game_money_change_4', 'enchant_count_1',
       'enchant_count_2', 'enchant_count_3', 'enchant_count_4',
       'combat_pledge_cnt_1', 'combat_pledge_cnt_2',
       'combat

In [25]:
X_train.to_csv('X_train_ver2.csv', index = False)
X_test.to_csv('X_test_ver2.csv', index = False)

# test1, test2 전처리

In [19]:
test1_1 = pd.read_csv('/home/user3/anaconda3/빅콘테스트/test1_activity.csv', engine = 'python')
test1_2 = pd.read_csv('/home/user3/anaconda3/빅콘테스트/test1_combat.csv', engine = 'python')
test1_3 = pd.read_csv('/home/user3/anaconda3/빅콘테스트/test1_pledge.csv', engine = 'python')
test1_4 = pd.read_csv('/home/user3/anaconda3/빅콘테스트/test1_payment.csv', engine = 'python')
test1_5 = pd.read_csv('/home/user3/anaconda3/빅콘테스트/test1_trade.csv', engine = 'python')

In [20]:
test2_1 = pd.read_csv('/home/user3/anaconda3/빅콘테스트/test2_activity.csv', engine = 'python')
test2_2 = pd.read_csv('/home/user3/anaconda3/빅콘테스트/test2_combat.csv', engine = 'python')
test2_3 = pd.read_csv('/home/user3/anaconda3/빅콘테스트/test2_pledge.csv', engine = 'python')
test2_4 = pd.read_csv('/home/user3/anaconda3/빅콘테스트/test2_payment.csv', engine = 'python')
test2_5 = pd.read_csv('/home/user3/anaconda3/빅콘테스트/test2_trade.csv', engine = 'python')

In [21]:
test1_id = test1_1['acc_id'].unique()
test1_id.sort()
test1_id

array([     7,     15,     16, ..., 130465, 130466, 130474])

In [22]:
test2_id = test2_1['acc_id'].unique()
test2_id.sort()
test2_id

array([     1,      3,      9, ..., 130458, 130467, 130471])

In [23]:
X_test1 = preprocess(test1_1, test1_2, test1_3, test1_4, test1_5, test1_id)

playtime done
npc_kill done
solo_exp done
party_exp done
quest_exp done
rich_monster done
death done
revive done
exp_recovery done
fishing done
private_shop done
game_money_change done
enchant_count done
pledge_cnt done
random_attacker_cnt done
random_defender_cnt done
temp_cnt done
same_pledge_cnt done
etc_cnt done
num_opponent done


/home/user3/.conda/envs/hun/lib/python3.6/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


play_char_cnt done
combat_char_cnt done
pledge_combat_cnt done
random_attacker_cnt done
random_defender_cnt done
same_pledge_cnt done
temp_cnt done
etc_cnt done
combat_play_time done


/home/user3/.conda/envs/hun/lib/python3.6/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)
/home/user3/.conda/envs/hun/lib/python3.6/site-packages/ipykernel_launcher.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/user3/.conda/envs/hun/lib/python3.6/site-packages/ipykernel_launcher.py:181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexin

In [24]:
X_test1.head()

,acc_id,level,playtime_1,playtime_2,playtime_3,playtime_4,npc_kill_1,npc_kill_2,npc_kill_3,npc_kill_4,...,target_type_1_week3_etc,target_type_1_week4_etc,source_weapon,target_weapon,source_armor,target_armor,source_accessory,target_accessory,source_spell,target_spell
0,7.0,11,0.000000,0.294923,0.159165,0.798163,0.000000,0.148018,0.069954,0.501503,...,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0
1,15.0,14,21.716599,22.599026,20.206876,21.182930,4.131994,5.162035,4.697367,4.216479,...,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0
2,16.0,9,0.000000,0.000000,0.032769,0.049154,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0
3,18.0,0,0.000000,0.000000,17.655562,22.353257,0.000000,0.000000,0.000000,0.000000,...,2.396984e-06,0.000000e+00,0,0,1,1,0,0,0,0
4,19.0,17,21.267193,20.138997,20.689051,22.044291,24.798383,26.081203,12.165168,16.821645,...,2.396984e-08,4.793968e-08,1,1,0,1,0,0,1,0


In [25]:
X_test1.columns.values

array(['acc_id', 'level', 'playtime_1', 'playtime_2', 'playtime_3',
       'playtime_4', 'npc_kill_1', 'npc_kill_2', 'npc_kill_3',
       'npc_kill_4', 'solo_exp_1', 'solo_exp_2', 'solo_exp_3',
       'solo_exp_4', 'party_exp_1', 'party_exp_2', 'party_exp_3',
       'party_exp_4', 'quest_exp_1', 'quest_exp_2', 'quest_exp_3',
       'quest_exp_4', 'rich_monster_1', 'rich_monster_2',
       'rich_monster_3', 'rich_monster_4', 'death_1', 'death_2',
       'death_3', 'death_4', 'revive_1', 'revive_2', 'revive_3',
       'revive_4', 'exp_recovery_1', 'exp_recovery_2', 'exp_recovery_3',
       'exp_recovery_4', 'fishing_1', 'fishing_2', 'fishing_3',
       'fishing_4', 'private_shop_1', 'private_shop_2', 'private_shop_3',
       'private_shop_4', 'game_money_change_1', 'game_money_change_2',
       'game_money_change_3', 'game_money_change_4', 'enchant_count_1',
       'enchant_count_2', 'enchant_count_3', 'enchant_count_4',
       'combat_pledge_cnt_1', 'combat_pledge_cnt_2',
       'combat

In [26]:
X_test2 = preprocess(test2_1, test2_2, test2_3, test2_4, test2_5, test2_id)

playtime done
npc_kill done
solo_exp done
party_exp done
quest_exp done
rich_monster done
death done
revive done
exp_recovery done
fishing done
private_shop done
game_money_change done
enchant_count done
pledge_cnt done
random_attacker_cnt done
random_defender_cnt done
temp_cnt done
same_pledge_cnt done
etc_cnt done
num_opponent done
play_char_cnt done
combat_char_cnt done
pledge_combat_cnt done
random_attacker_cnt done
random_defender_cnt done
same_pledge_cnt done
temp_cnt done
etc_cnt done
combat_play_time done


/home/user3/.conda/envs/hun/lib/python3.6/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)
/home/user3/.conda/envs/hun/lib/python3.6/site-packages/ipykernel_launcher.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/user3/.conda/envs/hun/lib/python3.6/site-packages/ipykernel_launcher.py:181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexin

In [27]:
X_test2.head()

,acc_id,level,playtime_1,playtime_2,playtime_3,playtime_4,npc_kill_1,npc_kill_2,npc_kill_3,npc_kill_4,...,target_type_1_week3_etc,target_type_1_week4_etc,source_weapon,target_weapon,source_armor,target_armor,source_accessory,target_accessory,source_spell,target_spell
0,1.0,16,22.030247,22.105148,21.857038,22.559235,14.669980,14.596309,14.468568,14.551025,...,0.000000,0.000000e+00,0,0,0,0,0,0,0,0
1,3.0,16,0.524307,0.936262,0.730284,0.503241,0.038525,0.015545,0.000000,0.038525,...,0.000000,0.000000e+00,0,0,0,0,1,0,0,0
2,9.0,11,0.004681,0.000000,0.000000,0.077242,0.000000,0.000000,0.000000,0.015883,...,0.000000,2.396984e-08,1,1,1,1,1,1,0,0
3,14.0,16,0.000000,0.000000,2.481095,4.077421,0.000000,0.000000,0.054746,0.161197,...,0.000016,4.408053e-05,0,0,1,1,1,1,1,1
4,26.0,17,8.910874,7.422218,8.274216,7.209218,2.049269,0.925956,0.586326,0.213240,...,0.000013,4.530300e-06,1,1,1,1,1,1,1,1


In [28]:
X_test2.columns.values

array(['acc_id', 'level', 'playtime_1', 'playtime_2', 'playtime_3',
       'playtime_4', 'npc_kill_1', 'npc_kill_2', 'npc_kill_3',
       'npc_kill_4', 'solo_exp_1', 'solo_exp_2', 'solo_exp_3',
       'solo_exp_4', 'party_exp_1', 'party_exp_2', 'party_exp_3',
       'party_exp_4', 'quest_exp_1', 'quest_exp_2', 'quest_exp_3',
       'quest_exp_4', 'rich_monster_1', 'rich_monster_2',
       'rich_monster_3', 'rich_monster_4', 'death_1', 'death_2',
       'death_3', 'death_4', 'revive_1', 'revive_2', 'revive_3',
       'revive_4', 'exp_recovery_1', 'exp_recovery_2', 'exp_recovery_3',
       'exp_recovery_4', 'fishing_1', 'fishing_2', 'fishing_3',
       'fishing_4', 'private_shop_1', 'private_shop_2', 'private_shop_3',
       'private_shop_4', 'game_money_change_1', 'game_money_change_2',
       'game_money_change_3', 'game_money_change_4', 'enchant_count_1',
       'enchant_count_2', 'enchant_count_3', 'enchant_count_4',
       'combat_pledge_cnt_1', 'combat_pledge_cnt_2',
       'combat

In [29]:
X_test1.to_csv('X_test1_ver2.csv', index = False)
X_test2.to_csv('X_test2_ver2.csv', index = False)

In [30]:
test1_1['acc_id'].unique()

array([ 31609,  95442, 117641, ...,  87075,  26124,  40300])

# 추가 전처리

### 직업 변수 제거

In [635]:
X_train = pd.read_csv('X_train_ver2.csv')
X_test = pd.read_csv('X_test_ver2.csv')
X_test1 = pd.read_csv('X_test1_ver2.csv')
X_test2 = pd.read_csv('X_test2_ver2.csv')

In [636]:
var_list = []
for i in range(0, len(X_train.columns)):
    if('class' in X_train.columns[i]):
        var_list.append(i)

In [637]:
X_train = X_train.drop(X_train.columns[var_list], axis = 1)
X_test = X_test.drop(X_test.columns[var_list], axis = 1)
X_test1 = X_test1.drop(X_test1.columns[var_list], axis = 1)
X_test2 = X_test2.drop(X_test2.columns[var_list], axis = 1)

In [638]:
X_train.to_csv('X_train_ver2_non_class.csv', index = False)
X_test.to_csv('X_test_ver2_non_class.csv', index = False)
X_test1.to_csv('X_test1_ver2_non_class.csv', index = False)
X_test2.to_csv('X_test2_ver2_non_class.csv', index = False)

### 플레이 시간 변동

In [948]:
X_train = pd.read_csv('X_train_ver2_non_class.csv')
X_test = pd.read_csv('X_test_ver2_non_class.csv')
X_test1 = pd.read_csv('X_test1_ver2_non_class.csv')
X_test2 = pd.read_csv('X_test2_ver2_non_class.csv')

In [949]:
X_train['playtime_fluctuation'] = (X_train['playtime_4'] + 1) / (X_train['playtime_3'] + 1)
X_test['playtime_fluctuation'] = (X_test['playtime_4'] + 1) / (X_test['playtime_3'] + 1)
X_test1['playtime_fluctuation'] = (X_test1['playtime_4'] + 1) / (X_test1['playtime_3'] + 1)
X_test2['playtime_fluctuation'] = (X_test2['playtime_4'] + 1) / (X_test2['playtime_3'] + 1)

In [950]:
X_train.to_csv('X_train_ver2_fluctuation.csv', index = False)
X_test.to_csv('X_test_ver2_fluctuation.csv', index = False)
X_test1.to_csv('X_test1_ver2_fluctuation.csv', index = False)
X_test2.to_csv('X_test2_ver2_fluctuation.csv', index = False)

### 유저별 접속일수, 서버수, 캐릭터수

In [951]:
X_train = pd.read_csv('X_train_ver2_fluctuation.csv')
X_test = pd.read_csv('X_test_ver2_fluctuation.csv')
X_test1 = pd.read_csv('X_test1_ver2_fluctuation.csv')
X_test2 = pd.read_csv('X_test2_ver2_fluctuation.csv')

In [952]:
index_list = []
for i in range(0, len(train1)):
    if(train1['acc_id'][i] in train_id):
        index_list.append(i)            
train1_sub = train1.loc[index_list]

In [953]:
index_list = []
for i in range(0, len(train1)):
    if(train1['acc_id'][i] in test_id):
        index_list.append(i)            
test1_sub = train1.loc[index_list]

In [954]:
index_list = []
for i in range(0, len(test1_1)):
    if(test1_1['acc_id'][i] in test1_id):
        index_list.append(i)            
test1_1_sub = test1_1.loc[index_list]

In [955]:
index_list = []
for i in range(0, len(test2_1)):
    if(test2_1['acc_id'][i] in test2_id):
        index_list.append(i)            
test2_1_sub = test2_1.loc[index_list]

In [956]:
def count_num(data, variable, user_id, version = 1):
    '''
    유저 아이디 별로 원하는 변수에 대한 unique값 count
    원하는 user_id에 대한 sub data가 있는 경우 version = 1, 없는 경우 version = 2
    version = 2 의 경우 subset 과정으로 시간이 더 오래 소요
    '''
    
    if(version == 1):
        data_sub = data
    else:
        index_list = []
        for i in range(0, len(data)):
            if(data['acc_id'][i] in user_id):
                index_list.append(i)            
        data_sub = data.loc[index_list]
    
    variable_unique = data_sub.groupby('acc_id')[variable].unique()
    variable_num = np.zeros(len(user_id))

    for i in range(0, len(user_id)):
        variable_num[i] = len(variable_unique[user_id[i]])
        
    return variable_num

In [957]:
day_num = count_num(data = train1_sub, variable = 'day', user_id = train_id, version = 1)
server_num = count_num(data = train1_sub, variable = 'server', user_id = train_id, version = 1)
char_num = count_num(data = train1_sub, variable = 'char_id', user_id = train_id, version = 1)

X_train['day_num'] = day_num
X_train['server_num'] = server_num
X_train['char_num'] = char_num

In [958]:
day_num = count_num(data = test1_sub, variable = 'day', user_id = test_id, version = 1)
server_num = count_num(data = test1_sub, variable = 'server', user_id = test_id, version = 1)
char_num = count_num(data = test1_sub, variable = 'char_id', user_id = test_id, version = 1)

X_test['day_num'] = day_num
X_test['server_num'] = server_num
X_test['char_num'] = char_num

In [959]:
day_num = count_num(data = test1_1_sub, variable = 'day', user_id = test1_id, version = 1)
server_num = count_num(data = test1_1_sub, variable = 'server', user_id = test1_id, version = 1)
char_num = count_num(data = test1_1_sub, variable = 'char_id', user_id = test1_id, version = 1)

X_test1['day_num'] = day_num
X_test1['server_num'] = server_num
X_test1['char_num'] = char_num

In [960]:
day_num = count_num(data = test2_1_sub, variable = 'day', user_id = test2_id, version = 1)
server_num = count_num(data = test2_1_sub, variable = 'server', user_id = test2_id, version = 1)
char_num = count_num(data = test2_1_sub, variable = 'char_id', user_id = test2_id, version = 1)

X_test2['day_num'] = day_num
X_test2['server_num'] = server_num
X_test2['char_num'] = char_num

In [961]:
X_train.to_csv('X_train_ver2_fluctuation.csv', index = False)
X_test.to_csv('X_test_ver2_fluctuation.csv', index = False)
X_test1.to_csv('X_test1_ver2_fluctuation.csv', index = False)
X_test2.to_csv('X_test2_ver2_fluctuation.csv', index = False)

### 이상치 대체

In [962]:
X_train = pd.read_csv('X_train_ver2_fluctuation.csv')
X_test = pd.read_csv('X_test_ver2_fluctuation.csv')
X_test1 = pd.read_csv('X_test1_ver2_fluctuation.csv')
X_test2 = pd.read_csv('X_test2_ver2_fluctuation.csv')

#### npc_kill, solo_exp, rich_monster, combat_same_pledge_cnt

In [963]:
X_train['npc_kill_1'] = (X_train['npc_kill_2'] + X_train['npc_kill_3'] + X_train['npc_kill_4']) / 3
X_train['solo_exp_1'] = (X_train['solo_exp_2'] + X_train['solo_exp_3'] + X_train['solo_exp_4']) / 3
X_train['rich_monster_2'] = (X_train['rich_monster_1'] + X_train['rich_monster_3'] + X_train['rich_monster_4']) / 3
# X_train['combat_same_pledge_cnt_1'] = (X_train['combat_same_pledge_cnt_2'] + X_train['combat_same_pledge_cnt_3'] + X_train['combat_same_pledge_cnt_4']) / 3

In [964]:
X_test['npc_kill_1'] = (X_test['npc_kill_2'] + X_test['npc_kill_3'] + X_test['npc_kill_4']) / 3
X_test['solo_exp_1'] = (X_test['solo_exp_2'] + X_test['solo_exp_3'] + X_test['solo_exp_4']) / 3
X_test['rich_monster_2'] = (X_test['rich_monster_1'] + X_test['rich_monster_3'] + X_test['rich_monster_4']) / 3
# X_test['combat_same_pledge_cnt_1'] = (X_test['combat_same_pledge_cnt_2'] + X_test['combat_same_pledge_cnt_3'] + X_test['combat_same_pledge_cnt_4']) / 3

##### game_money_change

In [965]:
train1_sub_day = train1_sub[(train1_sub['day'] == 1) | (train1_sub['day'] == 8) | (train1_sub['day'] == 22)]
test1_sub_day = test1_sub[(test1_sub['day'] == 1) | (test1_sub['day'] == 8) | (test1_sub['day'] == 22)]

In [966]:
val1 = train1_sub_day.groupby('acc_id')['game_money_change'].sum() / 3
val2 = train1_sub[train1_sub['day'] == 16].groupby('acc_id')['game_money_change'].sum()
val3 = train1_sub[train1_sub['day'] == 17].groupby('acc_id')['game_money_change'].sum()
val4 = train1_sub[train1_sub['day'] == 18].groupby('acc_id')['game_money_change'].sum()
val5 = train1_sub[train1_sub['day'] == 19].groupby('acc_id')['game_money_change'].sum()
val6 = train1_sub[train1_sub['day'] == 20].groupby('acc_id')['game_money_change'].sum()
val7 = train1_sub[train1_sub['day'] == 21].groupby('acc_id')['game_money_change'].sum()

df1 = pd.DataFrame(val1)
df2 = pd.DataFrame(val2)
df3 = pd.DataFrame(val3)
df4 = pd.DataFrame(val4)
df5 = pd.DataFrame(val5)
df6 = pd.DataFrame(val6)
df7 = pd.DataFrame(val7)

df = pd.DataFrame(train_id)
df.columns = ['acc_id']

df = pd.merge(df, df1, how = 'left', on = 'acc_id')
df = pd.merge(df, df2, how = 'left', on = 'acc_id')
df = pd.merge(df, df3, how = 'left', on = 'acc_id')
df = pd.merge(df, df4, how = 'left', on = 'acc_id')
df = pd.merge(df, df5, how = 'left', on = 'acc_id')
df = pd.merge(df, df6, how = 'left', on = 'acc_id')
df = pd.merge(df, df7, how = 'left', on = 'acc_id')

df = df.fillna(0)
game_money_change_sum = df.iloc[:, 1:].sum(axis = 1)
X_train['game_money_change_3'] = game_money_change_sum

In [967]:
val1 = test1_sub_day.groupby('acc_id')['game_money_change'].sum() / 3
val2 = test1_sub[test1_sub['day'] == 16].groupby('acc_id')['game_money_change'].sum()
val3 = test1_sub[test1_sub['day'] == 17].groupby('acc_id')['game_money_change'].sum()
val4 = test1_sub[test1_sub['day'] == 18].groupby('acc_id')['game_money_change'].sum()
val5 = test1_sub[test1_sub['day'] == 19].groupby('acc_id')['game_money_change'].sum()
val6 = test1_sub[test1_sub['day'] == 20].groupby('acc_id')['game_money_change'].sum()
val7 = test1_sub[test1_sub['day'] == 21].groupby('acc_id')['game_money_change'].sum()

df1 = pd.DataFrame(val1)
df2 = pd.DataFrame(val2)
df3 = pd.DataFrame(val3)
df4 = pd.DataFrame(val4)
df5 = pd.DataFrame(val5)
df6 = pd.DataFrame(val6)
df7 = pd.DataFrame(val7)

df = pd.DataFrame(test_id)
df.columns = ['acc_id']

df = pd.merge(df, df1, how = 'left', on = 'acc_id')
df = pd.merge(df, df2, how = 'left', on = 'acc_id')
df = pd.merge(df, df3, how = 'left', on = 'acc_id')
df = pd.merge(df, df4, how = 'left', on = 'acc_id')
df = pd.merge(df, df5, how = 'left', on = 'acc_id')
df = pd.merge(df, df6, how = 'left', on = 'acc_id')
df = pd.merge(df, df7, how = 'left', on = 'acc_id')

df = df.fillna(0)
game_money_change_sum = df.iloc[:, 1:].sum(axis = 1)
X_test['game_money_change_3'] = game_money_change_sum

#### enchant_count

In [968]:
val1 = train1_sub_day.groupby('acc_id')['enchant_count'].sum() / 3
val2 = train1_sub[train1_sub['day'] == 16].groupby('acc_id')['enchant_count'].sum()
val3 = train1_sub[train1_sub['day'] == 17].groupby('acc_id')['enchant_count'].sum()
val4 = train1_sub[train1_sub['day'] == 18].groupby('acc_id')['enchant_count'].sum()
val5 = train1_sub[train1_sub['day'] == 19].groupby('acc_id')['enchant_count'].sum()
val6 = train1_sub[train1_sub['day'] == 20].groupby('acc_id')['enchant_count'].sum()
val7 = train1_sub[train1_sub['day'] == 21].groupby('acc_id')['enchant_count'].sum()

df1 = pd.DataFrame(val1)
df2 = pd.DataFrame(val2)
df3 = pd.DataFrame(val3)
df4 = pd.DataFrame(val4)
df5 = pd.DataFrame(val5)
df6 = pd.DataFrame(val6)
df7 = pd.DataFrame(val7)

df = pd.DataFrame(train_id)
df.columns = ['acc_id']

df = pd.merge(df, df1, how = 'left', on = 'acc_id')
df = pd.merge(df, df2, how = 'left', on = 'acc_id')
df = pd.merge(df, df3, how = 'left', on = 'acc_id')
df = pd.merge(df, df4, how = 'left', on = 'acc_id')
df = pd.merge(df, df5, how = 'left', on = 'acc_id')
df = pd.merge(df, df6, how = 'left', on = 'acc_id')
df = pd.merge(df, df7, how = 'left', on = 'acc_id')

df = df.fillna(0)
enchant_count_sum = df.iloc[:, 1:].sum(axis = 1)
X_train['enchant_count_3'] = enchant_count_sum

In [969]:
val1 = test1_sub_day.groupby('acc_id')['enchant_count'].sum() / 3
val2 = test1_sub[test1_sub['day'] == 16].groupby('acc_id')['enchant_count'].sum()
val3 = test1_sub[test1_sub['day'] == 17].groupby('acc_id')['enchant_count'].sum()
val4 = test1_sub[test1_sub['day'] == 18].groupby('acc_id')['enchant_count'].sum()
val5 = test1_sub[test1_sub['day'] == 19].groupby('acc_id')['enchant_count'].sum()
val6 = test1_sub[test1_sub['day'] == 20].groupby('acc_id')['enchant_count'].sum()
val7 = test1_sub[test1_sub['day'] == 21].groupby('acc_id')['enchant_count'].sum()

df1 = pd.DataFrame(val1)
df2 = pd.DataFrame(val2)
df3 = pd.DataFrame(val3)
df4 = pd.DataFrame(val4)
df5 = pd.DataFrame(val5)
df6 = pd.DataFrame(val6)
df7 = pd.DataFrame(val7)

df = pd.DataFrame(test_id)
df.columns = ['acc_id']

df = pd.merge(df, df1, how = 'left', on = 'acc_id')
df = pd.merge(df, df2, how = 'left', on = 'acc_id')
df = pd.merge(df, df3, how = 'left', on = 'acc_id')
df = pd.merge(df, df4, how = 'left', on = 'acc_id')
df = pd.merge(df, df5, how = 'left', on = 'acc_id')
df = pd.merge(df, df6, how = 'left', on = 'acc_id')
df = pd.merge(df, df7, how = 'left', on = 'acc_id')

df = df.fillna(0)
enchant_count_sum = df.iloc[:, 1:].sum(axis = 1)
X_test['enchant_count_3'] = enchant_count_sum

#### combat_temp_cnt

In [970]:
index_list = []
for i in range(0, len(train2)):
    if(train2['acc_id'][i] in train_id):
        index_list.append(i)            
train2_sub = train2.loc[index_list]

In [971]:
index_list = []
for i in range(0, len(train2)):
    if(train2['acc_id'][i] in test_id):
        index_list.append(i)            
test2_sub = train2.loc[index_list]

In [972]:
index_list = []
for i in range(0, len(test1_2)):
    if(test1_2['acc_id'][i] in test1_id):
        index_list.append(i)            
test1_2_sub = test1_2.loc[index_list]

In [973]:
index_list = []
for i in range(0, len(test2_2)):
    if(test2_2['acc_id'][i] in test2_id):
        index_list.append(i)            
test2_2_sub = test2_2.loc[index_list]

In [974]:
train2_sub_day_1 = train2_sub[(train2_sub['day'] == 8) | (train2_sub['day'] == 15)]
train2_sub_day_2 = train2_sub[(train2_sub['day'] == 9) | (train2_sub['day'] == 16)]
train2_sub_day_3 = train2_sub[(train2_sub['day'] == 10) | (train2_sub['day'] == 17)]
train2_sub_day_4 = train2_sub[(train2_sub['day'] == 11) | (train2_sub['day'] == 18)]
train2_sub_day_5 = train2_sub[(train2_sub['day'] == 12) | (train2_sub['day'] == 19)]
train2_sub_day_6 = train2_sub[(train2_sub['day'] == 13) | (train2_sub['day'] == 20)]
train2_sub_day_7 = train2_sub[(train2_sub['day'] == 14) | (train2_sub['day'] == 21)]

test2_sub_day_1 = test2_sub[(test2_sub['day'] == 8) | (test2_sub['day'] == 15)]
test2_sub_day_2 = test2_sub[(test2_sub['day'] == 9) | (test2_sub['day'] == 16)]
test2_sub_day_3 = test2_sub[(test2_sub['day'] == 10) | (test2_sub['day'] == 17)]
test2_sub_day_4 = test2_sub[(test2_sub['day'] == 11) | (test2_sub['day'] == 18)]
test2_sub_day_5 = test2_sub[(test2_sub['day'] == 12) | (test2_sub['day'] == 19)]
test2_sub_day_6 = test2_sub[(test2_sub['day'] == 13) | (test2_sub['day'] == 20)]
test2_sub_day_7 = test2_sub[(test2_sub['day'] == 14) | (test2_sub['day'] == 21)]

In [975]:
val1 = train2_sub_day_1.groupby('acc_id')['temp_cnt'].sum() / 2
val2 = train2_sub_day_2.groupby('acc_id')['temp_cnt'].sum() / 2
val3 = train2_sub_day_3.groupby('acc_id')['temp_cnt'].sum() / 2
val4 = train2_sub[train2_sub['day'] == 4].groupby('acc_id')['temp_cnt'].sum()
val5 = train2_sub[train2_sub['day'] == 5].groupby('acc_id')['temp_cnt'].sum()
val6 = train2_sub[train2_sub['day'] == 6].groupby('acc_id')['temp_cnt'].sum()
val7 = train2_sub[train2_sub['day'] == 7].groupby('acc_id')['temp_cnt'].sum()

df1 = pd.DataFrame(val1)
df2 = pd.DataFrame(val2)
df3 = pd.DataFrame(val3)
df4 = pd.DataFrame(val4)
df5 = pd.DataFrame(val5)
df6 = pd.DataFrame(val6)
df7 = pd.DataFrame(val7)

df = pd.DataFrame(train_id)
df.columns = ['acc_id']

df = pd.merge(df, df1, how = 'left', on = 'acc_id')
df = pd.merge(df, df2, how = 'left', on = 'acc_id')
df = pd.merge(df, df3, how = 'left', on = 'acc_id')
df = pd.merge(df, df4, how = 'left', on = 'acc_id')
df = pd.merge(df, df5, how = 'left', on = 'acc_id')
df = pd.merge(df, df6, how = 'left', on = 'acc_id')
df = pd.merge(df, df7, how = 'left', on = 'acc_id')

df = df.fillna(0)
combat_temp_cnt_sum = df.iloc[:, 1:].sum(axis = 1)
# X_train['combat_temp_cnt_1'] = combat_temp_cnt_sum

In [976]:
val1 = train2_sub[train2_sub['day'] == 22].groupby('acc_id')['temp_cnt'].sum()
val2 = train2_sub_day_2.groupby('acc_id')['temp_cnt'].sum() / 2
val3 = train2_sub_day_3.groupby('acc_id')['temp_cnt'].sum() / 2
val4 = train2_sub_day_4.groupby('acc_id')['temp_cnt'].sum() / 2
val5 = train2_sub_day_5.groupby('acc_id')['temp_cnt'].sum() / 2
val6 = train2_sub_day_6.groupby('acc_id')['temp_cnt'].sum() / 2
val7 = train2_sub_day_7.groupby('acc_id')['temp_cnt'].sum() / 2

df1 = pd.DataFrame(val1)
df2 = pd.DataFrame(val2)
df3 = pd.DataFrame(val3)
df4 = pd.DataFrame(val4)
df5 = pd.DataFrame(val5)
df6 = pd.DataFrame(val6)
df7 = pd.DataFrame(val7)

df = pd.DataFrame(train_id)
df.columns = ['acc_id']

df = pd.merge(df, df1, how = 'left', on = 'acc_id')
df = pd.merge(df, df2, how = 'left', on = 'acc_id')
df = pd.merge(df, df3, how = 'left', on = 'acc_id')
df = pd.merge(df, df4, how = 'left', on = 'acc_id')
df = pd.merge(df, df5, how = 'left', on = 'acc_id')
df = pd.merge(df, df6, how = 'left', on = 'acc_id')
df = pd.merge(df, df7, how = 'left', on = 'acc_id')

df = df.fillna(0)
combat_temp_cnt_sum = df.iloc[:, 1:].sum(axis = 1)
# X_train['combat_temp_cnt_4'] = combat_temp_cnt_sum

In [977]:
val1 = test2_sub_day_1.groupby('acc_id')['temp_cnt'].sum() / 2
val2 = test2_sub_day_2.groupby('acc_id')['temp_cnt'].sum() / 2
val3 = test2_sub_day_3.groupby('acc_id')['temp_cnt'].sum() / 2
val4 = test2_sub[test2_sub['day'] == 4].groupby('acc_id')['temp_cnt'].sum()
val5 = test2_sub[test2_sub['day'] == 5].groupby('acc_id')['temp_cnt'].sum()
val6 = test2_sub[test2_sub['day'] == 6].groupby('acc_id')['temp_cnt'].sum()
val7 = test2_sub[test2_sub['day'] == 7].groupby('acc_id')['temp_cnt'].sum()

df1 = pd.DataFrame(val1)
df2 = pd.DataFrame(val2)
df3 = pd.DataFrame(val3)
df4 = pd.DataFrame(val4)
df5 = pd.DataFrame(val5)
df6 = pd.DataFrame(val6)
df7 = pd.DataFrame(val7)

df = pd.DataFrame(test_id)
df.columns = ['acc_id']

df = pd.merge(df, df1, how = 'left', on = 'acc_id')
df = pd.merge(df, df2, how = 'left', on = 'acc_id')
df = pd.merge(df, df3, how = 'left', on = 'acc_id')
df = pd.merge(df, df4, how = 'left', on = 'acc_id')
df = pd.merge(df, df5, how = 'left', on = 'acc_id')
df = pd.merge(df, df6, how = 'left', on = 'acc_id')
df = pd.merge(df, df7, how = 'left', on = 'acc_id')

df = df.fillna(0)
combat_temp_cnt_sum = df.iloc[:, 1:].sum(axis = 1)
# X_test['combat_temp_cnt_1'] = combat_temp_cnt_sum

In [978]:
val1 = test2_sub[test2_sub['day'] == 22].groupby('acc_id')['temp_cnt'].sum()
val2 = test2_sub_day_2.groupby('acc_id')['temp_cnt'].sum() / 2
val3 = test2_sub_day_3.groupby('acc_id')['temp_cnt'].sum() / 2
val4 = test2_sub_day_4.groupby('acc_id')['temp_cnt'].sum() / 2
val5 = test2_sub_day_5.groupby('acc_id')['temp_cnt'].sum() / 2
val6 = test2_sub_day_6.groupby('acc_id')['temp_cnt'].sum() / 2
val7 = test2_sub_day_7.groupby('acc_id')['temp_cnt'].sum() / 2

df1 = pd.DataFrame(val1)
df2 = pd.DataFrame(val2)
df3 = pd.DataFrame(val3)
df4 = pd.DataFrame(val4)
df5 = pd.DataFrame(val5)
df6 = pd.DataFrame(val6)
df7 = pd.DataFrame(val7)

df = pd.DataFrame(test_id)
df.columns = ['acc_id']

df = pd.merge(df, df1, how = 'left', on = 'acc_id')
df = pd.merge(df, df2, how = 'left', on = 'acc_id')
df = pd.merge(df, df3, how = 'left', on = 'acc_id')
df = pd.merge(df, df4, how = 'left', on = 'acc_id')
df = pd.merge(df, df5, how = 'left', on = 'acc_id')
df = pd.merge(df, df6, how = 'left', on = 'acc_id')
df = pd.merge(df, df7, how = 'left', on = 'acc_id')

df = df.fillna(0)
combat_temp_cnt_sum = df.iloc[:, 1:].sum(axis = 1)
# X_test['combat_temp_cnt_4'] = combat_temp_cnt_sum

In [979]:
print(X_train['npc_kill_1'].mean())
print(X_train['npc_kill_2'].mean())
print(X_train['npc_kill_3'].mean())
print(X_train['npc_kill_4'].mean())

3.024934882407761
3.0049064227822044
2.66868607277297
3.4012121516681084


In [980]:
print(X_train['solo_exp_1'].mean())
print(X_train['solo_exp_2'].mean())
print(X_train['solo_exp_3'].mean())
print(X_train['solo_exp_4'].mean())

1.5769237900304192
1.659560420524457
1.4892014827699314
1.5820094667968683


In [981]:
print(X_train['rich_monster_1'].mean())
print(X_train['rich_monster_2'].mean())
print(X_train['rich_monster_3'].mean())
print(X_train['rich_monster_4'].mean())

0.9780666666666666
0.9790555555555555
0.9796666666666667
0.9794333333333334


In [982]:
print(X_train['game_money_change_1'].mean())
print(X_train['game_money_change_2'].mean())
print(X_train['game_money_change_3'].mean())
print(X_train['game_money_change_4'].mean())

-0.004584662079693132
0.01387940129471397
-0.04116805343539996
-0.0026584526810235066


In [983]:
print(X_train['enchant_count_1'].mean())
print(X_train['enchant_count_2'].mean())
print(X_train['enchant_count_3'].mean())
print(X_train['enchant_count_4'].mean())

0.08323203533946184
0.12309948083819566
0.14991092079348448
0.19793836975687143


In [984]:
print(X_train['combat_same_pledge_cnt_1'].mean())
print(X_train['combat_same_pledge_cnt_2'].mean())
print(X_train['combat_same_pledge_cnt_3'].mean())
print(X_train['combat_same_pledge_cnt_4'].mean())

1.1788690776870454
0.476300302203092
0.37752739788505385
0.4265005742937883


In [985]:
print(X_train['combat_temp_cnt_1'].mean())
print(X_train['combat_temp_cnt_2'].mean())
print(X_train['combat_temp_cnt_3'].mean())
print(X_train['combat_temp_cnt_4'].mean())

0.632153265870426
0.6105175437029999
0.6445942058898327
0.8053669706136328


In [986]:
X_train.to_csv('X_train_ver2_fluctuation.csv', index = False)
X_test.to_csv('X_test_ver2_fluctuation.csv', index = False)
X_test1.to_csv('X_test1_ver2_fluctuation.csv', index = False)
X_test2.to_csv('X_test2_ver2_fluctuation.csv', index = False)

### 불필요한 주차별 데이터 28일 평균으로

In [987]:
X_train = pd.read_csv('X_train_ver2_fluctuation.csv')
X_test = pd.read_csv('X_test_ver2_fluctuation.csv')
X_test1 = pd.read_csv('X_test1_ver2_fluctuation.csv')
X_test2 = pd.read_csv('X_test2_ver2_fluctuation.csv')

In [988]:
def week_to_mean(data, variable, drop = True):
    '''
    주차별 합으로 이루어진 변수를 28일 동안의 평균으로 변환
    주차별 변수는 삭제
    '''
    
    var_list = []
    for i in range(0, len(data.columns)):
        if(variable in data.columns[i]):
            var_list.append(i)
        
    column_name = variable + '_mean'
    data[column_name] = data.iloc[:, var_list].mean(axis = 1)
    if(drop == True):
        data = data.drop(data.columns[var_list], axis = 1)
    
    return data

In [989]:
variable_list = ['death', 'revive', 'exp_recovery', 'enchant_count', 'combat_random_attacker_cnt',
                'combat_random_defender_cnt', 'combat_same_pledge_cnt']

In [990]:
for variable in variable_list:
    X_train = week_to_mean(data = X_train, variable = variable)

In [991]:
for variable in variable_list:
    X_test = week_to_mean(data = X_test, variable = variable)

In [992]:
for variable in variable_list:
    X_test1 = week_to_mean(data = X_test1, variable = variable)

In [993]:
for variable in variable_list:
    X_test2 = week_to_mean(data = X_test2, variable = variable)

In [994]:
X_train.to_csv('X_train_ver2_fluctuation.csv', index = False)
X_test.to_csv('X_test_ver2_fluctuation.csv', index = False)
X_test1.to_csv('X_test1_ver2_fluctuation.csv', index = False)
X_test2.to_csv('X_test2_ver2_fluctuation.csv', index = False)

In [995]:
X_test2.head()

,acc_id,level,playtime_1,playtime_2,playtime_3,playtime_4,npc_kill_1,npc_kill_2,npc_kill_3,npc_kill_4,...,day_num,server_num,char_num,death_mean,revive_mean,exp_recovery_mean,enchant_count_mean,combat_random_attacker_cnt_mean,combat_random_defender_cnt_mean,combat_same_pledge_cnt_mean
0,1.0,16,22.030247,22.105148,21.857038,22.559235,14.669980,14.596309,14.468568,14.551025,...,28.0,1.0,2.0,0.676180,0.000000,0.0,0.0,0.0,20.112096,0.000000
1,3.0,16,0.524307,0.936262,0.730284,0.503241,0.038525,0.015545,0.000000,0.038525,...,21.0,5.0,9.0,0.000000,0.000000,0.0,0.0,0.0,0.295766,0.000000
2,9.0,11,0.004681,0.000000,0.000000,0.077242,0.000000,0.000000,0.000000,0.015883,...,4.0,1.0,4.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
3,14.0,16,0.000000,0.000000,2.481095,4.077421,0.000000,0.000000,0.054746,0.161197,...,12.0,2.0,8.0,0.245883,0.247337,0.0,0.0,0.0,0.000000,0.000000
4,26.0,17,8.910874,7.422218,8.274216,7.209218,2.049269,0.925956,0.586326,0.213240,...,28.0,4.0,6.0,7.376504,7.420107,0.0,0.0,0.0,0.887298,6.199136


### 레벨 변화

In [807]:
X_train = pd.read_csv('X_train_ver2_fluctuation.csv')
X_test = pd.read_csv('X_test_ver2_fluctuation.csv')
X_test1 = pd.read_csv('X_test1_ver2_fluctuation.csv')
X_test2 = pd.read_csv('X_test2_ver2_fluctuation.csv')

In [808]:
def level_diff(data, user_id):
    '''
    28일 동안의 레별 변화를 구함
    '''
    
    char_list = []
    char_matrix = data.groupby(['acc_id', 'char_id']).count()['day']
    for i in user_id:
        char_list.append(np.argmax(char_matrix[i]))
        
    level_diff_list = []
    for i in char_list:
        user_char = data[data['char_id'] == i]
        char_level_diff = user_char['level'].values[-1] - user_char['level'].values[0]
        level_diff_list.append(char_level_diff)
        
    return np.array(level_diff_list)

In [809]:
train_level_diff = level_diff(data = train2_sub, user_id = train_id)
X_train['level_diff'] = train_level_diff

In [810]:
test_level_diff = level_diff(data = test2_sub, user_id = test_id)
X_test['level_diff'] = test_level_diff

In [814]:
test1_level_diff = level_diff(data = test1_2_sub, user_id = test1_id)
X_test1['level_diff'] = test1_level_diff

In [815]:
test2_level_diff = level_diff(data = test2_2_sub, user_id = test2_id)
X_test2['level_diff'] = test2_level_diff

In [820]:
X_train.to_csv('X_train_ver2_fluctuation.csv', index = False)
X_test.to_csv('X_test_ver2_fluctuation.csv', index = False)
X_test1.to_csv('X_test1_ver2_fluctuation.csv', index = False)
X_test2.to_csv('X_test2_ver2_fluctuation.csv', index = False)

### 아데나 변동량 절댓값

In [877]:
X_train = pd.read_csv('X_train_ver2_fluctuation.csv')
X_test = pd.read_csv('X_test_ver2_fluctuation.csv')
X_test1 = pd.read_csv('X_test1_ver2_fluctuation.csv')
X_test2 = pd.read_csv('X_test2_ver2_fluctuation.csv')

In [878]:
train1_sub['abs_game_money_change'] = abs(train1_sub['game_money_change'])
test1_sub['abs_game_money_change'] = abs(test1_sub['game_money_change'])
test1_1_sub['abs_game_money_change'] = abs(test1_1_sub['game_money_change'])
test2_1_sub['abs_game_money_change'] = abs(test2_1_sub['game_money_change'])

In [879]:
week1, week2, week3, week4 = divide_by_week(data = train1_sub, variable = 'abs_game_money_change', user_id = train_id)
X_train['abs_game_money_change_1'] = week1
X_train['abs_game_money_change_2'] = week2
X_train['abs_game_money_change_3'] = week3
X_train['abs_game_money_change_4'] = week4

abs_game_money_change done


In [880]:
week1, week2, week3, week4 = divide_by_week(data = test1_sub, variable = 'abs_game_money_change', user_id = test_id)
X_test['abs_game_money_change_1'] = week1
X_test['abs_game_money_change_2'] = week2
X_test['abs_game_money_change_3'] = week3
X_test['abs_game_money_change_4'] = week4

abs_game_money_change done


In [881]:
week1, week2, week3, week4 = divide_by_week(data = test1_1_sub, variable = 'abs_game_money_change', user_id = test1_id)
X_test1['abs_game_money_change_1'] = week1
X_test1['abs_game_money_change_2'] = week2
X_test1['abs_game_money_change_3'] = week3
X_test1['abs_game_money_change_4'] = week4

abs_game_money_change done


In [882]:
week1, week2, week3, week4 = divide_by_week(data = test2_1_sub, variable = 'abs_game_money_change', user_id = test2_id)
X_test2['abs_game_money_change_1'] = week1
X_test2['abs_game_money_change_2'] = week2
X_test2['abs_game_money_change_3'] = week3
X_test2['abs_game_money_change_4'] = week4

abs_game_money_change done


In [888]:
X_train.to_csv('X_train_ver2_fluctuation.csv', index = False)
X_test.to_csv('X_test_ver2_fluctuation.csv', index = False)
X_test1.to_csv('X_test1_ver2_fluctuation.csv', index = False)
X_test2.to_csv('X_test2_ver2_fluctuation.csv', index = False)

### 불필요한 주차별 데이터 28일 통계

In [391]:
X_train = pd.read_csv('X_train_ver2_fluctuation.csv')
X_test = pd.read_csv('X_test_ver2_fluctuation.csv')
X_test1 = pd.read_csv('X_test1_ver2_fluctuation.csv')
X_test2 = pd.read_csv('X_test2_ver2_fluctuation.csv')

In [392]:
def week_to_summary(data, variable, drop = True):
    '''
    주차별 합으로 이루어진 변수를 28일 동안의 통계값으로 변환
    주차별 변수는 삭제
    '''
    
    var_list = []
    for i in range(0, len(data.columns)):
        if(variable in data.columns[i]):
            var_list.append(i)
        
    column_name = variable + '_sum'
    data[column_name] = data.iloc[:, var_list].sum(axis = 1)
    column_name = variable + '_mean'
    data[column_name] = data.iloc[:, var_list].mean(axis = 1)
    column_name = variable + '_std'
    data[column_name] = data.iloc[:, var_list].std(axis = 1)
    if(drop == True):
        data = data.drop(data.columns[var_list], axis = 1)
    
    return data

In [393]:
# variable_list = ['death', 'revive', 'exp_recovery', 'enchant_count', 'combat_random_attacker_cnt', 'combat_random_defender_cnt', 'combat_same_pledge_cnt']

variabe_list = ['playtime', 'npc_kill', 'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive', 'exp_recovery',
                'fishing', 'private_shop', 'game_money_change', 'enchant_count', 'combat_pledge_cnt', 'combat_random_attacker_cnt',
                'combat_random_defender_cnt', 'combat_temp_cnt', 'combat_same_pledge_cnt', 'combat_etc_cnt', 'combat_num_opponent']

In [394]:
for variable in variabe_list:
    X_train = week_to_summary(data = X_train, variable = variable)

In [395]:
for variable in variabe_list:
    X_test = week_to_summary(data = X_test, variable = variable)

In [396]:
for variable in variabe_list:
    X_test1 = week_to_summary(data = X_test1, variable = variable)

In [397]:
for variable in variabe_list:
    X_test2 = week_to_summary(data = X_test2, variable = variable)

In [398]:
X_train.to_csv('X_train_ver2_fluctuation.csv', index = False)
X_test.to_csv('X_test_ver2_fluctuation.csv', index = False)
X_test1.to_csv('X_test1_ver2_fluctuation.csv', index = False)
X_test2.to_csv('X_test2_ver2_fluctuation.csv', index = False)

In [401]:
X_test.head()

,acc_id,level,amount_cnt,amount_mean,server_group_0,server_group_1,server_group_2,pledge_group_0.0,pledge_group_1.0,pledge_group_2.0,...,combat_temp_cnt_std,combat_same_pledge_cnt_sum,combat_same_pledge_cnt_mean,combat_same_pledge_cnt_std,combat_etc_cnt_sum,combat_etc_cnt_mean,combat_etc_cnt_std,combat_num_opponent_sum,combat_num_opponent_mean,combat_num_opponent_std
0,20.0,17,3.0,0.298844,1,0,0,0,0,1,...,0.239864,0.0,0.0,0.0,16.441283,4.110321,4.414889,10.107262,2.526815,2.144474
1,62.0,16,4.0,1.072845,0,1,0,0,0,0,...,0.908110,0.0,0.0,0.0,4.054015,1.013504,1.952655,14.621184,3.655296,3.466946
2,81.0,16,2.0,0.941123,0,1,0,0,0,0,...,1.491534,0.0,0.0,0.0,2.927900,0.731975,0.599421,4.710180,1.177545,0.768501
3,86.0,0,0.0,0.000000,0,1,0,1,0,0,...,0.276971,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.196258,0.049064,0.056655
4,125.0,16,0.0,0.000000,0,1,0,0,1,0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### 주차별 편차

In [937]:
X_train = pd.read_csv('X_train_ver2_fluctuation.csv')
X_test = pd.read_csv('X_test_ver2_fluctuation.csv')
X_test1 = pd.read_csv('X_test1_ver2_fluctuation.csv')
X_test2 = pd.read_csv('X_test2_ver2_fluctuation.csv')

In [938]:
def week_std(data, variable):
    '''
    주차별 데이터간의 표준편차
    '''
    
    var_list = []
    for i in range(0, len(data.columns)):
        if(variable in data.columns[i]):
            var_list.append(i)
        
    column_name = variable + '_std'
    data[column_name] = data.iloc[:, var_list].std(axis = 1)
    
    return data

In [939]:
variable_list = ['playtime', 'npc_kill', 'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'fishing', 'private_shop', 'game_money_change']

In [940]:
for variable in variable_list:
    X_train = week_std(data = X_train, variable = variable)

In [941]:
for variable in variable_list:
    X_test = week_std(data = X_test, variable = variable)

In [942]:
for variable in variable_list:
    X_test1 = week_std(data = X_test1, variable = variable)

In [943]:
for variable in variable_list:
    X_test2 = week_std(data = X_test2, variable = variable)

In [947]:
X_train.to_csv('X_train_ver2_fluctuation.csv', index = False)
X_test.to_csv('X_test_ver2_fluctuation.csv', index = False)
X_test1.to_csv('X_test1_ver2_fluctuation.csv', index = False)
X_test2.to_csv('X_test2_ver2_fluctuation.csv', index = False)

In [944]:
X_train.head()

,acc_id,level,playtime_1,playtime_2,playtime_3,playtime_4,npc_kill_1,npc_kill_2,npc_kill_3,npc_kill_4,...,combat_same_pledge_cnt_mean,playtime_std,npc_kill_std,solo_exp_std,party_exp_std,quest_exp_std,rich_monster_std,fishing_std,private_shop_std,game_money_change_std
0,2.0,0,22.919696,22.980553,22.659883,22.973531,0.000000,0.000000,0.000000,0.000000,...,0.000000,9.781512,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.189828,0.258903
1,5.0,15,0.000000,0.589845,0.213000,5.313287,1.078817,0.310229,0.162887,2.763336,...,0.000000,2.742303,1.192653,0.617452,0.347643,0.055527,2.624669,1.721454,0.000000,0.063718
2,8.0,16,4.987936,4.889629,4.653223,4.267014,1.454832,1.325063,1.444694,1.594739,...,0.000000,1.707731,0.110328,0.449333,0.000000,0.237551,0.471405,0.447836,0.000000,0.111758
3,17.0,16,5.406914,9.048973,7.831832,6.176989,1.508565,1.328780,1.362575,1.834339,...,3.099568,3.155919,0.230770,1.026825,2.072678,0.197221,0.471405,1.996786,0.000000,0.043515
4,21.0,16,6.048253,7.363701,4.966870,3.452466,2.130600,4.602406,0.654252,1.135141,...,0.000000,2.549657,1.758822,2.384015,0.088881,4.575611,1.414214,1.385748,0.000000,0.041284
